In [1]:
# importar las librerias Pandas(Para manipular Datos) 
# y Openpyxl(Para crear graficos, tablas y realizar calculos en Excel)
import pandas as pd
import openpyxl

In [2]:
from openpyxl import load_workbook #para cargar la hoja de calculo
from openpyxl.styles import Font #para dar el estilo de las letras de la hoja
# Barchart para la creacion de graficos y Reference para definir rangos de celdas en una hoja de Excel que serán la base de datos del gráfico.
from openpyxl.chart import BarChart, Reference 
import string # para el tipo de datos

In [8]:
#Para leer el archivo Excel
excelFile = pd.read_excel("supermarket_sales.xlsx")

In [ ]:
#para ver las columnas que necesitamos para el informe
excelFile[["Gender", "Product line","Total"]]

,Gender,Product line,Total
0,Female,Health and beauty,548.9715
1,Female,Electronic accessories,80.2200
2,Male,Home and lifestyle,340.5255
3,Male,Health and beauty,489.0480
4,Male,Sports and travel,634.3785
...,...,...,...
995,Male,Health and beauty,42.3675
996,Female,Home and lifestyle,1022.4900
997,Male,Food and beverages,33.4320
998,Male,Home and lifestyle,69.1110


In [14]:
#Para crear la Tabla Dinamica del archivo excel, use la función .pivot_table()
reportTable = excelFile.pivot_table(
    index="Gender", #genero
    columns="Product line", #linea de producto
    values="Total", #el total de dinero gastado por hombres y mujeres en las diferentes líneas de productos 
    aggfunc="sum" #para la sumatoria de los puntos en las columnas agrupadas
).round(0) #para retorno nulo
display(reportTable) #muestra el resultado

Product line,Electronic accessories,Fashion accessories,Food and beverages,Health and beauty,Home and lifestyle,Sports and travel
Gender,,,,,,
Female,27102.0,30437.0,33171.0,18561.0,30037.0,28575.0
Male,27236.0,23868.0,22974.0,30633.0,23825.0,26548.0


In [ ]:
#use la funcion .to_excel() para exportar el informe en un archivo excel
reportTable.to_excel(
    "report_2024.xlsx", #nombre del archivo excel
    sheet_name="ReportPivotTable", #nombre de la hoja de excel
    startrow=4 #genera los datos despues de 4 filas
)

In [17]:
#creare referencias de filas y columnas
wb = load_workbook("report_2024.xlsx") #para cargar el archivo que queremos usar
sheet = wb["ReportPivotTable"] #para cargar la hoja que queremos usar



In [22]:
#refencia del archivo de planilla
minColumn = wb.active.min_column
maxColumn = wb.active.max_column

minRow = wb.active.min_row
maxRow = wb.active.max_row

In [16]:
#Creare una funcion para automatizar todo el proceso

# importar las librerias Pandas(Para manipular Datos) 
# y Openpyxl(Para crear graficos, tablas y realizar calculos en Excel)
import pandas as pd
import openpyxl
from openpyxl import load_workbook #para cargar la hoja de calculo
from openpyxl.styles import Font #para dar el estilo de las letras de la hoja
# Barchart para la creacion de graficos y Reference para definir rangos de celdas en una hoja de Excel que serán la base de datos del gráfico.
from openpyxl.chart import BarChart, Reference 
import string # para el tipo de datos

#delaro la funcion 
def automate_excelReport(file_name):
    # para leer el archivo en Excel
    excelFile = pd.read_excel(file_name)

    #Para crear la Tabla Dinamica del archivo excel, use la función .pivot_table()
    reportTable = excelFile.pivot_table(
        index=["Gender","City"], #genero
        columns="Product line", #linea de producto
        values="Total", #el total de dinero gastado por hombres y mujeres en las diferentes líneas de productos 
        aggfunc=["sum","mean"] #para la sumatoria de los puntos en las columnas agrupadas
    ).round(0) #para retorno nulo
    display(reportTable) #muestra el resultado


    #dividir el mes y la extencion del nombre del archivo
    monthAndExtention = file_name.split("_")[1]

    #use la funcion .to_excel() para exportar el informe en un archivo excel
    reportTable.to_excel(
        f"report__{monthAndExtention}", #nombre del archivo excel
        sheet_name="Report", #nombre de la hoja de excel
        startrow=4 #genera los datos despues de 4 filas
    )
   

    # cargar la carpeta de trabajo y selecionar hoja de cálculo
    wb = load_workbook(f"report__{monthAndExtention}") #para cargar el archivo que queremos usar
    sheet = wb["Report"] #para cargar la hoja que queremos usar

    #refencia de celdas del archivo orginal, es decir el de supermarket_sales.xlsx
    minColumn = wb.active.min_column
    maxColumn = wb.active.max_column

    minRow = wb.active.min_row + 4
    maxRow = wb.active.max_row

    #agregando un grafico
    graphic = BarChart()
    #para el encabezado
    data = Reference(sheet,min_col=minColumn+1,min_row=minRow,max_col=maxColumn,max_row=maxRow)
    #para los datos despues de encabezado
    categories = Reference(sheet,min_col=minColumn,min_row=minRow,max_col=maxColumn,max_row=maxRow)

    graphic.add_data(data, titles_from_data=True)
    graphic.set_categories(categories)

    #para la ubicacion del grafico
    sheet.add_chart(graphic, "B20")
    #para el titulo
    BarChart.title = "Ventas por Lineas de Productos"
    #para el estilo
    BarChart.style = 8

    #aplicar formulas, creando una lista del las letras del abcdario
    alphabetList = list(string.ascii_uppercase)

    alphabetList = list(string.ascii_uppercase)
    excelAlphabet = alphabetList[minColumn - 1:maxColumn]

    for i in excelAlphabet[1:]:
        sheet[f"{i}{maxRow + 1}"] = f"=SUM({i}{minRow}:{i}{maxRow})"
        sheet[f"{i}{maxRow + 1}"].number_format = '#,##0.00'

    # Etiqueta de "Total" en la columna A
    sheet[f"A{maxRow + 1}"] = "Total"
    sheet[f"A{maxRow + 1}"].font = Font(bold=True)
    
    #ejecutar "get" en el nombre del mes
    monthName = monthAndExtention.split(".")[0] 

    #formateando el reporte
    sheet["A1"] = "Reprote de Ventas"
    #sheet["A2"] = monthName.title()
    sheet["A1"].font = Font("Arial", bold=True, size=25)
    #sheet["A2"].font = Font("Arial", bold=True, size=15)

    #Para guardar el archivo
    wb.save(f"report__{monthAndExtention}")
    #Para validar
    print("Reporte ejecutado!")
    #return para el resultado
    #return

#llamar a la funcion 
automate_excelReport("supermarket_sales.xlsx")

sum                      \
Product line     Electronic accessories Fashion accessories   
Gender City                                                   
Female Mandalay                  8167.0              9081.0   
       Naypyitaw                 8969.0             11520.0   
       Yangon                    9966.0              9836.0   
Male   Mandalay                  8885.0              7332.0   
       Naypyitaw                10000.0             10040.0   
       Yangon                    8351.0              6496.0   

                                                                          \
Product line     Food and beverages Health and beauty Home and lifestyle   
Gender City                                                                
Female Mandalay             10510.0            6400.0             9557.0   
       Naypyitaw            15661.0            6446.0             7843.0   
       Yangon                7000.0            5715.0            12637.0   
Male   Mandalay              4705.0           13580.0             7992.0   
       Naypyitaw             8106.0           10170.0             6053.0   
       Yangon               10163.0            6883.0             9780.0   

                                                     mean                      \
Product line     Sports and travel Electronic accessories Fashion accessories   
Gender City                                                                     
Female Mandalay             9213.0                  292.0               275.0   
       Naypyitaw           11247.0                  320.0               329.0   
       Yangon               8115.0                  356.0               351.0   
Male   Mandalay            10775.0                  329.0               253.0   
       Naypyitaw            4515.0                  370.0               335.0   
       Yangon              11258.0                  261.0               282.0   

                                                                          \
Product line     Food and beverages Health and beauty Home and lifestyle   
Gender City                                                                
Female Mandalay               362.0             320.0              434.0   
       Naypyitaw              412.0             280.0              314.0   
       Yangon                 304.0             272.0              395.0   
Male   Mandalay               224.0             412.0              285.0   
       Naypyitaw              289.0             351.0              303.0   
       Yangon                 290.0             265.0              296.0   

                                    
Product line     Sports and travel  
Gender City                         
Female Mandalay              307.0  
       Naypyitaw             388.0  
       Yangon                280.0  
Male   Mandalay              337.0  
       Naypyitaw             282.0  
       Yangon                375.0

Reporte ejecutado!


In [18]:
#Generacion de reportes automaticamente
import schedule
import time
import os

def GenerateReportAuto():
    print("Report Generation In Process...")
    automate_excelReport("supermarket_sales.xlsx")
    print(f"Report generated at: {os.getcwd()}")

schedule.every().tuesday.at("20:30").do(GenerateReportAuto)

print("Waiting for the time to Generate the report")

while True:
    schedule.run_pending()
    time.sleep(1)



Waiting for the time to Generate the report
Report Generation In Process...


sum                      \
Product line     Electronic accessories Fashion accessories   
Gender City                                                   
Female Mandalay                  8167.0              9081.0   
       Naypyitaw                 8969.0             11520.0   
       Yangon                    9966.0              9836.0   
Male   Mandalay                  8885.0              7332.0   
       Naypyitaw                10000.0             10040.0   
       Yangon                    8351.0              6496.0   

                                                                          \
Product line     Food and beverages Health and beauty Home and lifestyle   
Gender City                                                                
Female Mandalay             10510.0            6400.0             9557.0   
       Naypyitaw            15661.0            6446.0             7843.0   
       Yangon                7000.0            5715.0            12637.0   
Male   Mandalay              4705.0           13580.0             7992.0   
       Naypyitaw             8106.0           10170.0             6053.0   
       Yangon               10163.0            6883.0             9780.0   

                                                     mean                      \
Product line     Sports and travel Electronic accessories Fashion accessories   
Gender City                                                                     
Female Mandalay             9213.0                  292.0               275.0   
       Naypyitaw           11247.0                  320.0               329.0   
       Yangon               8115.0                  356.0               351.0   
Male   Mandalay            10775.0                  329.0               253.0   
       Naypyitaw            4515.0                  370.0               335.0   
       Yangon              11258.0                  261.0               282.0   

                                                                          \
Product line     Food and beverages Health and beauty Home and lifestyle   
Gender City                                                                
Female Mandalay               362.0             320.0              434.0   
       Naypyitaw              412.0             280.0              314.0   
       Yangon                 304.0             272.0              395.0   
Male   Mandalay               224.0             412.0              285.0   
       Naypyitaw              289.0             351.0              303.0   
       Yangon                 290.0             265.0              296.0   

                                    
Product line     Sports and travel  
Gender City                         
Female Mandalay              307.0  
       Naypyitaw             388.0  
       Yangon                280.0  
Male   Mandalay              337.0  
       Naypyitaw             282.0  
       Yangon                375.0

Reporte ejecutado!
Report generated at: c:\Users\nandy\OneDrive\Escritorio\PythonTest\AutomatedTask\automateExcelReport


KeyboardInterrupt: 